In [ ]:
# ==========================================
# CELL 1: SETUP & IMPORTS
# ==========================================
import timesfm  # pip install git+https://github.com/google-research/timesfm.git
import pandas as pd
import numpy as np
import torch
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Generic Hardware Check
device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device.upper()}")

# ==========================================
# CELL 2: LOAD MODEL
# ==========================================
# Using the 500M model from Hugging Face
CHECKPOINT_REPO = "google/timesfm-2.0-500m-pytorch"

print(f"Loading TimesFM from {CHECKPOINT_REPO}...")

tfm = timesfm.TimesFm(
    context_len=2048,
    horizon_len=14,         # M4 Daily Horizon
    input_patch_len=32,
    output_patch_len=128,
    num_layers=50,
    model_dims=1280,
    backend=device
)

tfm.load_from_checkpoint(repo_id=CHECKPOINT_REPO)
print("✅ Model loaded.")

# ==========================================
# CELL 3: LOAD DATA
# ==========================================
# Ensure you have 'm4_daily.csv' in your data folder
# Columns required: 'unique_id', 'ds', 'y'
DATA_PATH = "../data/m4_daily.csv"

print("Loading M4 Data...")
try:
    df = pd.read_csv(DATA_PATH)
    df['ds'] = pd.to_datetime(df['ds'])
    print(f"✅ Loaded {len(df)} rows. Unique Series: {df['unique_id'].nunique()}")
except FileNotFoundError:
    print(f"❌ Error: {DATA_PATH} not found.")

# ==========================================
# CELL 4: ZERO-SHOT INFERENCE
# ==========================================
print("Running Forecast (Freq='D')...")

forecast_df = tfm.forecast_on_df(
    inputs=df,
    freq="D",
    value_name="y",
    num_jobs=-1
)
print("✅ Inference Complete.")

# ==========================================
# CELL 5: ROBUST METRICS (Correct MASE)
# ==========================================
def calculate_metrics_robust(y_true, y_pred, y_train_hist, seasonality=1):
    """
    Calculates metrics with correct MASE denominator (Training History).
    """
    # Standard Errors
    mae = np.mean(np.abs(y_true - y_pred))
    mse = np.mean((y_true - y_pred) ** 2)
    rmse = np.sqrt(mse)

    # sMAPE
    epsilon = 1e-10
    smape = 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + epsilon))

    # Correct MASE Denominator
    if len(y_train_hist) > seasonality:
        naive_mae = np.mean(np.abs(y_train_hist[seasonality:] - y_train_hist[:-seasonality]))
    elif len(y_train_hist) > 1:
        naive_mae = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        naive_mae = epsilon

    mase = mae / (naive_mae + epsilon)

    return {"MAE": mae, "RMSE": rmse, "sMAPE": smape, "MASE": mase}

# --- Evaluation Loop ---
print("Calculating Metrics...")
metrics_list = []

# Group by Series (Assuming 'df' contains history + test truth)
for uid, group in df.groupby('unique_id'):
    pred_row = forecast_df[forecast_df['unique_id'] == uid]
    if pred_row.empty: continue

    y_hist = group['y'].values
    # Hold out last 14 steps for truth
    y_true = y_hist[-14:]
    y_hist_train = y_hist[:-14]
    y_pred = pred_row['timesfm'].values[:14]

    if len(y_true) == len(y_pred):
        m = calculate_metrics_robust(y_true, y_pred, y_hist_train, seasonality=1)
        m['unique_id'] = uid
        metrics_list.append(m)

metrics_df = pd.DataFrame(metrics_list)
print("\n" + "="*40)
print(f"FINAL M4 RESULTS (n={len(metrics_df)})")
print("="*40)
print(f"Mean MASE:  {metrics_df['MASE'].mean():.4f}")
print(f"Mean sMAPE: {metrics_df['sMAPE'].mean():.4f}%")
print("="*40)

In [ ]:
# [Setup & Imports same as above]
# ...

# ==========================================
# CELL 2: LOAD MODEL
# ==========================================
tfm = timesfm.TimesFm(
    context_len=2048,
    horizon_len=96,         # Traffic Horizon
    input_patch_len=32,
    output_patch_len=128,
    num_layers=50,
    model_dims=1280,
    backend=device
)
tfm.load_from_checkpoint(repo_id="google/timesfm-2.0-500m-pytorch")

# ==========================================
# CELL 3: DATA & INFERENCE
# ==========================================
DATA_PATH = "../data/traffic.csv"
# ... Load Data Code ...

print("Running Forecast (Freq='H')...")
# Note: Hourly frequency
forecast_df = tfm.forecast_on_df(
    inputs=df,
    freq="H",
    value_name="y",
    num_jobs=-1
)

# ==========================================
# CELL 5: METRICS (Seasonality=24)
# ==========================================
# ... calculate_metrics_robust function definition ...

# Loop change:
    # y_true = y_hist[-96:]
    # y_hist_train = y_hist[:-96]
    # m = calculate_metrics_robust(y_true, y_pred, y_hist_train, seasonality=24) # 24 for Hourly Traffic

# ... Summary Print ...

In [ ]:
# Same structure as Traffic.
# Change DATA_PATH to "../data/etth1.csv"
# Keep Freq="H"
# Keep Seasonality=24 for MASE calculation.

In [ ]:
# Same structure as M4.
# Change DATA_PATH to "../data/exchange.csv"
# Change Horizon to 96
# Keep Freq="D"
# Keep Seasonality=1 for MASE calculation.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

In [ ]:


def train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, full_df):

    print("\n--- Training XGBoost Model (Legacy API) ---")

    if len(X_train) == 0:
        print("ERROR: Training set is empty after cleaning.")
        return pd.DataFrame()

    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': 0.1,
        'max_depth': 8,
        'seed': 42,
        'nthread': -1,
        'tree_method': 'hist'
    }

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)

    eval_list = [(dval, 'validation')]

    start_time = time.time()
    bst = xgb.train(
        xgb_params, dtrain, num_boost_round=1000,
        evals=eval_list, early_stopping_rounds=50, verbose_eval=False
    )
    print(f"Training complete in {time.time() - start_time:.2f} seconds. ({bst.best_iteration} rounds)")

    # Prediction
    y_pred_test = bst.predict(dtest, iteration_range=(0, bst.best_iteration))

    # --- AGGREGATION (OPTIMIZED) ---
    print("\n--- Calculating Final Benchmarks (Optimized) ---")

    # 1. Prepare Results DataFrame
    results_df = full_df.loc[X_test.index, ['series_id', 'sales']].copy()
    results_df['y_pred'] = y_pred_test

    # 2. Pre-Calculate History for MASE Denominator (THE FIX)
    print("Pre-grouping historical data for fast lookup...")

    # Identify indices that are NOT in the test set (i.e., Train + Validation)
    history_indices = full_df.index.difference(X_test.index)
    history_df = full_df.loc[history_indices]

    # Create a GroupBy object - this creates the index map once!
    history_groups = history_df.groupby('series_id')['sales']

    metrics_list = []
    unique_ids = results_df['series_id'].unique()

    print(f"Aggregating metrics for {len(unique_ids)} unique series...")

    # 3. Fast Loop
    # Use groupby on results_df to iterate efficiently
    for series_id, series_res in results_df.groupby('series_id'):

        # Instant lookup from the pre-grouped history object
        if series_id in history_groups.groups:
            history_series = history_groups.get_group(series_id).values
        else:
            history_series = np.array([])

        metrics = calculate_metrics(
            series_res['sales'].values,
            series_res['y_pred'].values,
            history_series
        )
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"\nEvaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == '__main__':
    # ... (Main execution remains the same) ...
    X_train, y_train, X_val, y_val, X_test, y_test, full_df = load_and_prepare_m4_data()
    final_metrics_xgb = train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, full_df)
    print("\n✅ M4 XGBoost Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error

# --- CONFIGURATION ---
# Replace with your actual file paths
INPUT_TRAIN_FILE = 'M4_Daily_Train_PLX.csv'
INPUT_TEST_FILE = 'M4_Daily_Test_PLX.csv'
FORECAST_HORIZON = 14

def calculate_metrics(y_true, y_pred, y_train_hist):
    """
    Calculates standard M4 metrics.
    Crucially, for M4 Daily data, the MASE denominator is the
    Mean Absolute Error of the Naive 1 (Lag 1) forecast.
    """
    y_pred = np.maximum(0, y_pred) # Clip negatives if strictly positive domain

    # 1. Standard Errors
    mae = np.mean(np.abs(y_true - y_pred))
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    # 2. sMAPE (Symmetric Mean Absolute Percentage Error)
    # Formula: 200 * mean(|y - y_hat| / (|y| + |y_hat|))
    denom = np.abs(y_true) + np.abs(y_pred)
    mask = denom != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denom[mask]) if np.sum(mask) > 0 else 0.0

    # 3. MASE (Mean Absolute Scaled Error)
    # Numerator: MAE of the model
    # Denominator: MAE of Naive 1 Forecast on Training History (Lag 1)
    if len(y_train_hist) > 1:
        # np.diff calculates y[t] - y[t-1], which is the Naive 1 error
        naive_mae = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        naive_mae = 0.0

    # Avoid division by zero
    mase = mae / naive_mae if naive_mae > 1e-9 else np.nan

    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase}

def run_m4_naive1_baseline():
    print("Loading M4 Data")
    try:
        train_df = pd.read_csv(INPUT_TRAIN_FILE)
        test_df = pd.read_csv(INPUT_TEST_FILE)
    except FileNotFoundError:
        print("Error: Files not found. Please upload 'M4_Daily_Train_PLX.csv' and 'M4_Daily_Test_PLX.csv'")
        return

    # Normalize column names
    if 'value' in train_df.columns: train_df.rename(columns={'value': 'sales'}, inplace=True)
    if 'value' in test_df.columns: test_df.rename(columns={'value': 'sales'}, inplace=True)

    # Pre-group for speed
    print(f"Analyzing {train_df['series_id'].nunique()} series...")
    train_groups = train_df.groupby('series_id')['sales']
    test_groups = test_df.groupby('series_id')['sales']

    metrics_list = []
    unique_ids = test_df['series_id'].unique()

    start_time = time.time()

    for i, series_id in enumerate(unique_ids):
        # Get history and ground truth
        y_train = train_groups.get_group(series_id).values
        y_test = test_groups.get_group(series_id).values

        # --- NAIVE 1 FORECAST (for M4 Daily) ---
        # Logic: "Tomorrow will be the same as today"
        # We take the very last value of the training set and repeat it for the horizon.
        if len(y_train) > 0:
            last_value = y_train[-1]
            y_pred = np.full(FORECAST_HORIZON, last_value)
        else:
            y_pred = np.zeros(FORECAST_HORIZON)

        # Calculate metrics
        m = calculate_metrics(y_test, y_pred, y_train)
        m['unique_id'] = series_id
        metrics_list.append(m)

        if (i+1) % 1000 == 0:
            print(f"Processed {i+1} series...")

    # Aggregate
    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print("\n" + "="*40)
    print("FINAL M4 BASELINE RESULTS (NAIVE 1)")
    print("="*40)
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Total Time: {time.time() - start_time:.2f} seconds")
    print("="*40)

    return final_metrics

if __name__ == '__main__':
    run_m4_naive1_baseline()

In [ ]:
import pandas as pd
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# --- CONFIGURATION ---
INPUT_TRAIN_FILE = 'M4_Daily_Train_PLX.csv'
INPUT_TEST_FILE = 'M4_Daily_Test_PLX.csv'
LOOKBACK_WINDOW = 30
FORECAST_HORIZON = 14
BATCH_SIZE = 128
EPOCHS = 5  # Increased slightly as normalized data converges faster

# Set random seed
np.random.seed(42)
tf.random.set_seed(42)

# --- METRIC FUNCTIONS ---
def calculate_metrics(y_true, y_pred, y_train_hist):
    # Ensure no negatives for metrics that can't handle them (like sMAPE)
    y_pred = np.maximum(0, y_pred)

    mae = np.mean(np.abs(y_true - y_pred))
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)

    denom = np.abs(y_true) + np.abs(y_pred)
    mask = denom != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denom[mask]) if np.sum(mask) > 0 else 0.0

    # M4 Daily MASE denominator: Naive 1 (Lag 1)
    if len(y_train_hist) > 1:
        naive_mae = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        naive_mae = 0.0

    mase = mae / naive_mae if naive_mae > 1e-9 else np.nan

    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}

# --- DATA PREPARATION ---
def create_sequences(df, target_col, window, horizon):
    """Converts data into 3D arrays [Samples, Time Steps, 1] using the NORMALIZED column."""
    X, Y = [], []

    # Group by series_id
    for series_id, group in df.groupby('series_id'):
        data = group[target_col].values

        if len(data) < window + horizon:
            continue

        for i in range(len(data) - window - horizon + 1):
            X.append(data[i : i + window])
            Y.append(data[i + window : i + window + horizon])

    X = np.array(X).reshape(-1, window, 1)
    Y = np.array(Y)
    return X, Y

def load_and_prepare_m4_lstm():
    print("Loading M4 Data...")
    train_df = pd.read_csv(INPUT_TRAIN_FILE)
    test_df = pd.read_csv(INPUT_TEST_FILE)

    if 'value' in train_df.columns: train_df.rename(columns={'value': 'sales'}, inplace=True)
    if 'value' in test_df.columns: test_df.rename(columns={'value': 'sales'}, inplace=True)

    # --- NEW: NORMALIZATION STEP ---
    print("Normalizing Data per Series...")
    # Calculate Mean and Std for every series
    stats = train_df.groupby('series_id')['sales'].agg(['mean', 'std']).reset_index()

    # Handle constant series (std=0) to avoid division by zero
    stats['std'] = stats['std'].replace(0, 1.0)

    # Merge stats back to training data
    train_df = train_df.merge(stats, on='series_id', how='left')

    # Create 'norm_sales' column: (x - mean) / std
    train_df['norm_sales'] = (train_df['sales'] - train_df['mean']) / train_df['std']

    # Convert stats to a dictionary for easy lookup later during Inverse Transform
    stats_dict = stats.set_index('series_id').to_dict('index')

    print("Generating sequences from Normalized Data...")
    # Note: We pass 'norm_sales' as the target column
    X_all, Y_all = create_sequences(train_df, 'norm_sales', LOOKBACK_WINDOW, FORECAST_HORIZON)

    # Split sequences into Train/Val (90/10)
    split_idx = int(len(X_all) * 0.9)
    X_train, X_val = X_all[:split_idx], X_all[split_idx:]
    Y_train, Y_val = Y_all[:split_idx], Y_all[split_idx:]

    print(f"Sequence Shapes -> Train: {X_train.shape}, Val: {X_val.shape}")

    # Create Prediction Input (Last window of NORMALIZED history)
    print("Preparing Forecast Inputs...")
    X_pred_list = []
    valid_series_ids = []

    train_df_sorted = train_df.sort_values(['series_id', 'date'])

    for series_id, group in train_df_sorted.groupby('series_id'):
        hist = group['norm_sales'].values  # Grab normalized history

        if len(hist) >= LOOKBACK_WINDOW:
            X_pred_list.append(hist[-LOOKBACK_WINDOW:])
            valid_series_ids.append(series_id)
        else:
            padded = np.pad(hist, (LOOKBACK_WINDOW - len(hist), 0), 'constant')
            X_pred_list.append(padded)
            valid_series_ids.append(series_id)

    X_predict = np.array(X_pred_list).reshape(-1, LOOKBACK_WINDOW, 1)

    return X_train, Y_train, X_val, Y_val, X_predict, valid_series_ids, train_df, test_df, stats_dict

# --- MODEL & EXECUTION ---
def run_lstm_benchmark(X_train, Y_train, X_val, Y_val, X_predict, valid_series_ids, train_df, test_df, stats_dict):

    # 1. Build Model
    model = Sequential([
        LSTM(units=64, return_sequences=False, activation='tanh', input_shape=(LOOKBACK_WINDOW, 1)),
        Dense(FORECAST_HORIZON)
    ])
    model.compile(optimizer='adam', loss='mse')

    print("\n--- Training LSTM Model ---")
    start_time = time.time()

    model.fit(
        X_train, Y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, Y_val),
        verbose=1
    )

    # 2. Predict (Output is Normalized)
    print("Generating Forecasts...")
    Y_pred_norm = model.predict(X_predict)

    # 3. Metrics & Inverse Transform
    print("Calculating Metrics...")
    metrics_list = []

    pred_map_norm = dict(zip(valid_series_ids, Y_pred_norm))
    history_groups = train_df.groupby('series_id')['sales'] # Raw sales for Naive calc

    for series_id, group in test_df.groupby('series_id'):
        if series_id not in pred_map_norm:
            continue

        # Get Raw Truth
        y_true = group.sort_values('date')['sales'].values

        # Get Normalized Prediction
        y_pred_n = pred_map_norm[series_id]

        # --- NEW: INVERSE TRANSFORM ---
        # Actual = Norm * Std + Mean
        series_stats = stats_dict.get(series_id)
        if series_stats:
            y_pred = (y_pred_n * series_stats['std']) + series_stats['mean']
        else:
            y_pred = y_pred_n # Fallback (should not happen)

        # Handle length mismatch
        min_len = min(len(y_true), len(y_pred))
        y_true = y_true[:min_len]
        y_pred = y_pred[:min_len]

        if len(y_true) == 0: continue

        # Get Raw History for MASE
        if series_id in history_groups.groups:
            y_hist = history_groups.get_group(series_id).values
        else:
            y_hist = np.array([])

        m = calculate_metrics(y_true, y_pred, y_hist)
        m['unique_id'] = series_id
        metrics_list.append(m)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    end_time = time.time()

    print("\n" + "="*35)
    print("FINAL M4 DAILY BENCHMARK RESULTS (LSTM)")
    print("="*35)
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Total Time: {end_time - start_time:.2f} seconds")
    print("="*35)

if __name__ == '__main__':
    X_train, Y_train, X_val, Y_val, X_predict, valid_series_ids, train_df, test_df, stats_dict = load_and_prepare_m4_lstm()
    run_lstm_benchmark(X_train, Y_train, X_val, Y_val, X_predict, valid_series_ids, train_df, test_df, stats_dict)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import time

In [ ]:
# ==============================================================================
#   XGBOOST FORECASTING MODEL FOR HOURLY TRAFFIC DATA
# ==============================================================================
# This script trains an XGBoost model on the 'Traffic_Hourly_Train.csv' dataset.
# It uses a sliding window approach with lags (1h, 24h, 168h) to predict traffic.
#
# Metrics calculated: MAE, MSE, RMSE, sMAPE, MASE, Bias
# ==============================================================================

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'Traffic_Daily_Train.csv'
TEST_FILE = 'Traffic_Daily_Test.csv'

# Forecast Horizon: 168 hours (1 Week)
FORECAST_HORIZON = 168

# Validation size: Last 4 days (96 hours) of training data
VALIDATION_HOURS = 96

# Lag Shift: 1 means we use previous hour's data to predict current hour.
LAG_SHIFT = 1

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # Ensure non-negative predictions (traffic cannot be negative)
    y_pred = np.maximum(0, y_pred)

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # Calculate seasonal naive MAE (Traffic is hourly, dominant seasonality is Daily = 24)
    seasonality = 24
    if len(y_train_hist) > seasonality:
        naive_errors = np.abs(y_train_hist[seasonality:] - y_train_hist[:-seasonality])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Forecast Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}

def generate_features(df):
    """Generates time-series features for Hourly Traffic data."""

    # --- 1. TIME FEATURES ---
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['hour'] = df['date'].dt.hour # Critical for Hourly Data
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype('int64')

    # --- 2. CATEGORICAL FEATURES ---
    # Encode Series ID (Sensor Name)
    encoder = LabelEncoder()
    # Ensure input is uniform string to prevent TypeError
    df['series_id_encoded'] = encoder.fit_transform(df['series_id'].astype(str))

    # --- 3. LAG AND ROLLING WINDOW FEATURES ---
    # We define a helper to shift strictly within a Series ID
    def grouped_shift(col, shift_n):
        return df.groupby('series_id')[col].shift(shift_n)

    # Lags (Based on Hourly Logic)
    df['lag_1']   = grouped_shift('sales', LAG_SHIFT)
    df['lag_24']  = grouped_shift('sales', LAG_SHIFT + 23)  # 1 day ago
    df['lag_168'] = grouped_shift('sales', LAG_SHIFT + 167) # 1 week ago

    # Rolling Statistics
    # Rolling Mean of last 24 hours (shifted to avoid leakage)
    df['rolling_mean_24'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(24).mean()
    )
    df['rolling_std_24'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(24).std()
    )

    return df

def load_and_prepare_traffic_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found. Run the process_traffic_data.py script first.")
        return None

    # Load with low_memory=False to suppress mixed type warnings initially
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # --- CRITICAL FIX: Force series_id to string ---
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Rename value -> sales for consistency with internal logic
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Mark splits before merging
    df_train['is_test'] = False
    df_test['is_test'] = True

    # Combine for Feature Engineering (Preserve history at boundaries)
    full_df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
    full_df = full_df.sort_values(['series_id', 'date'])

    print("2. Generating Features...")
    full_df = generate_features(full_df)

    # --- SPLITTING ---
    # 1. Recover Test Set
    X_test_full = full_df[full_df['is_test'] == True].copy()

    # 2. Recover Train Set (All non-test data)
    train_full = full_df[full_df['is_test'] == False].copy()

    # 3. Create Validation Set from the END of the Train Set
    max_train_date = train_full['date'].max()
    val_start_date = max_train_date - pd.Timedelta(hours=VALIDATION_HOURS - 1)

    # Split Train/Val
    X_train_df = train_full[train_full['date'] < val_start_date]
    X_val_df = train_full[train_full['date'] >= val_start_date]

    # Define Feature Columns (Drop non-feature cols)
    drop_cols = ['sales', 'date', 'series_id', 'is_test', 'split_type']
    features = [c for c in full_df.columns if c not in drop_cols]

    print(f"   Features used: {features}")

    # Create Final Arrays
    X_train = X_train_df[features]
    y_train = X_train_df['sales']

    X_val = X_val_df[features]
    y_val = X_val_df['sales']

    X_test = X_test_full[features]
    y_test = X_test_full['sales']

    # Clean NaNs caused by lags (mostly in early train data)
    valid_indices = X_train.dropna().index
    X_train = X_train.loc[valid_indices]
    y_train = y_train.loc[valid_indices]

    print("\n--- Final Data Shapes ---")
    print(f"Train Set: {len(X_train)} rows")
    print(f"Val Set:   {len(X_val)} rows")
    print(f"Test Set:  {len(X_test)} rows")

    return X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full

def train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full_history):

    print("\n--- Training XGBoost Model ---")

    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': 0.05,
        'max_depth': 8,
        'seed': 42,
        'nthread': -1,
        'tree_method': 'hist'
    }
    num_round = 1000

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)

    eval_list = [(dtrain, 'train'), (dval, 'validation')]

    bst = xgb.train(
        xgb_params,
        dtrain,
        num_round,
        evals=eval_list,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    print(f"   Best Iteration: {bst.best_iteration}")

    # Prediction
    y_pred_test = bst.predict(dtest, iteration_range=(0, bst.best_iteration))

    # --- METRICS CALCULATION ---
    print("\n--- Calculating Final Benchmarks ---")

    results_df = X_test_full.copy()
    results_df['y_pred'] = y_pred_test
    # Ensure we use the actual sales column for truth, keeping index alignment
    results_df['sales'] = y_test

    metrics_list = []

    # Iterate over unique series to calculate MASE per series
    unique_series = results_df['series_id'].unique()

    for i, series_id in enumerate(unique_series):
        if i % 100 == 0: print(f"   Evaluated {i}/{len(unique_series)} series...", end='\r')

        series_results = results_df[results_df['series_id'] == series_id]

        # Use full training history (Train + Val) for MASE denominator
        history_series = train_full_history[train_full_history['series_id'] == series_id]['sales'].values

        metrics = calculate_metrics(
            series_results['sales'].values,
            series_results['y_pred'].values,
            history_series
        )
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"\nEvaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == '__main__':
    # Load data
    data_tuple = load_and_prepare_traffic_data()

    if data_tuple:
        X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full = data_tuple

        # Train and Eval
        train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full)

        print("\n✅ Traffic XGBoost Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'Traffic_Daily_Train.csv'
TEST_FILE = 'Traffic_Daily_Test.csv'

# PREDICTION SEASONALITY:
# Traffic is hourly. The strongest naive predictor is usually "same time last week"
# (captures weekend vs weekday differences).
SEASONALITY = 168 # 1 Week

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""
    y_pred = np.maximum(0, y_pred)

    # 1. MAE
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # NOTE: We use 24 (Daily) for the denominator to match the XGBoost script's scaling,
    # ensuring MASE is comparable across models.
    mase_seasonality = 24
    if len(y_train_hist) > mase_seasonality:
        naive_errors = np.abs(y_train_hist[mase_seasonality:] - y_train_hist[:-mase_seasonality])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}


def load_traffic_data():
    """Loads the pre-split Traffic CSVs."""
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"Error: {TRAIN_FILE} or {TEST_FILE} not found.")
        return None, None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    print(f"   Train Rows: {len(df_train)}")
    print(f"   Test Rows:  {len(df_test)}")

    return df_train, df_test


def run_snaive_benchmark(df_train, df_test):
    """
    Runs Seasonal Naive: Forecast = Last 168 hours (1 Week) of training, repeated.
    """
    metrics_list = []

    train_groups = df_train.groupby('series_id')
    test_groups = df_test.groupby('series_id')

    unique_ids = df_test['series_id'].unique()

    print(f"\n2. Running Seasonal Naive Benchmark (S={SEASONALITY}) on {len(unique_ids)} series...")
    start_time = time.time()

    for i, series_id in enumerate(unique_ids):
        if i > 0 and i % 100 == 0:
            print(f"   Processed {i}/{len(unique_ids)}...", end='\r')

        try:
            train_series = train_groups.get_group(series_id).sort_values('date')
            y_train = train_series['sales'].values

            test_series = test_groups.get_group(series_id).sort_values('date')
            y_test = test_series['sales'].values

            # Prediction Horizon
            horizon = len(y_test)

            # Forecast Logic: Repeat the last SEASONALITY window
            if len(y_train) < SEASONALITY:
                # Fallback if history is too short (rare for Traffic)
                y_pred = np.tile(y_train[-1:], horizon)
            else:
                # Grab last 168 hours
                last_season = y_train[-SEASONALITY:]
                repetitions = int(np.ceil(horizon / SEASONALITY))
                y_pred = np.tile(last_season, repetitions)[:horizon]

            metrics = calculate_metrics(y_test, y_pred, y_train)
            metrics['unique_id'] = series_id
            metrics_list.append(metrics)

        except KeyError:
            continue

    end_time = time.time()
    total_time = end_time - start_time

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"\n\n--- Final SNaive Results ({total_time:.2f}s) ---")
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == "__main__":
    df_train, df_test = load_traffic_data()

    if df_train is not None:
        run_snaive_benchmark(df_train, df_test)
        print("\n✅ Traffic SNaive Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.metrics import mean_squared_error

# --- CONFIGURATION ---
TRAIN_FILE = 'Traffic_Daily_Train.csv'
TEST_FILE = 'Traffic_Daily_Test.csv'

# Standard Hourly Benchmark Settings
LOOKBACK_WINDOW = 168   # 1 Week Context
FORECAST_HORIZON = 168  # 1 Week Prediction
VALIDATION_STEPS = 96   # 4 Days Validation Targets

# Generator Stride: Samples data every N hours to control epoch size
# Stride 1 = Use all data (Slowest/Most Data)
# Stride 24 = Use one sample per day per sensor (Fast/Good for benchmarks)
STRIDE = 1

np.random.seed(42)
tf.random.set_seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, mae_naive):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""
    y_pred = np.maximum(0, y_pred)

    # 1. MAE
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # Uses pre-calculated naive MAE (Daily Seasonality = 24 hours) passed as argument
    if mae_naive > 1e-9:
        mase = mae_forecast / mae_naive
    else:
        mase = np.nan

    # 6. Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}

# --- MEMORY-SAFE DATA GENERATOR ---

class TrafficGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, lookback, horizon, batch_size=256, stride=1, shuffle=True):
        super().__init__() # REQUIRED for Keras 3+ to avoid warnings
        self.lookback = lookback
        self.horizon = horizon
        self.batch_size = batch_size
        self.stride = stride
        self.shuffle = shuffle

        # Pre-process data into a dictionary of numpy arrays {series_id: array}
        # This keeps memory minimal (just raw data, no sequence duplication)
        self.data_dict = {}
        self.indices = []

        print("   Initializing Generator... (Log-Normalizing Data in Memory)")

        # Group by series and prepare data
        for series_id, group in df.groupby('series_id'):
            # Clean and Log Normalize ONCE here
            raw_vals = group['sales'].values.astype('float32')
            raw_vals = np.nan_to_num(raw_vals, nan=0.0, posinf=0.0, neginf=0.0)
            log_vals = np.log1p(np.maximum(0, raw_vals))

            self.data_dict[series_id] = log_vals

            # Calculate valid start indices
            # A valid start index i means we can take data[i : i+lookback] as X
            # and data[i+lookback : i+lookback+horizon] as Y
            usable_len = len(log_vals) - lookback - horizon + 1
            if usable_len > 0:
                for i in range(0, usable_len, stride):
                    self.indices.append((series_id, i))

        self.on_epoch_end()
        print(f"   Generator Ready: {len(self.indices)} samples.")

    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        # Generate one batch of data
        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]

        X = []
        Y = []

        for series_id, start_idx in batch_indices:
            series_data = self.data_dict[series_id]

            # X: Input Sequence
            X.append(series_data[start_idx : start_idx + self.lookback])

            # Y: Target Sequence
            Y.append(series_data[start_idx + self.lookback : start_idx + self.lookback + self.horizon])

        return np.array(X)[..., np.newaxis], np.array(Y)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

# --- LOADING ---

def load_and_prepare_traffic_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found.")
        return None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, usecols=['series_id', 'date', 'value'], low_memory=False)
    df_test = pd.read_csv(TEST_FILE, usecols=['series_id', 'date', 'value'], low_memory=False)

    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    df_train['date'] = pd.to_datetime(df_train['date'], errors='coerce')
    df_test['date'] = pd.to_datetime(df_test['date'], errors='coerce')

    df_train['sales'] = pd.to_numeric(df_train['sales'], errors='coerce')
    df_test['sales'] = pd.to_numeric(df_test['sales'], errors='coerce')

    df_train.dropna(subset=['date', 'sales'], inplace=True)
    df_test.dropna(subset=['date', 'sales'], inplace=True)

    df_train = df_train.sort_values(['series_id', 'date'])
    df_test = df_test.sort_values(['series_id', 'date'])

    # --- PREPARE PREDICTION INPUTS ---
    print("2. Preparing Prediction Inputs (X_predict)...")

    X_predict_list = []
    ids_list = []
    test_ids = df_test['series_id'].unique()

    # OPTIMIZATION: Group by series_id ONCE before the loop to make lookups O(1)
    # This prevents the slow repeated filtering of the large dataframe
    train_groups = df_train.groupby('series_id')

    for series_id in test_ids:
        # Fast retrieval using the groupby object
        if series_id not in train_groups.groups:
            continue

        series_hist = train_groups.get_group(series_id)['sales'].values.astype('float32')
        series_hist = np.nan_to_num(series_hist, nan=0.0)
        series_hist = np.log1p(np.maximum(0, series_hist))

        if len(series_hist) >= LOOKBACK_WINDOW:
            X_predict_list.append(series_hist[-LOOKBACK_WINDOW:])
            ids_list.append(series_id)
        else:
            padded = np.pad(series_hist, (LOOKBACK_WINDOW - len(series_hist), 0), 'constant')
            X_predict_list.append(padded)
            ids_list.append(series_id)

    X_predict = np.array(X_predict_list, dtype='float32').reshape(-1, LOOKBACK_WINDOW, 1)

    return df_train, df_test, X_predict, ids_list

def build_and_run_lstm(df_train, df_test, X_predict, ids_list):

    # 1. Pre-Calculate Naive MAE (MASE Denominator) to save RAM later
    print("3. Pre-calculating Naive Errors (MASE Denominator)...")
    naive_mae_dict = {}

    # Seasonality = 24 (Daily) for Hourly Data
    SEASONALITY = 24

    for series_id, group in df_train.groupby('series_id'):
        y_hist = group['sales'].values
        if len(y_hist) > SEASONALITY:
            naive_mae = np.mean(np.abs(y_hist[SEASONALITY:] - y_hist[:-SEASONALITY]))
        elif len(y_hist) > 1:
            naive_mae = np.mean(np.abs(np.diff(y_hist)))
        else:
            naive_mae = 0.0
        naive_mae_dict[series_id] = naive_mae

    print(f"   Computed Naive Errors for {len(naive_mae_dict)} series.")

    # 2. Setup Generators (The memory-safe part)
    print("4. Setting up Data Generators...")

    # --- FIX: OVERLAPPING SPLIT LOGIC ---
    # We determine the cutoff date for TRAINING targets.
    # To generate validation samples starting immediately after train,
    # the Val Generator needs LOOKBACK_WINDOW context from before the split.

    # Cutoff is essentially the start of the validation period targets
    max_train_date = df_train['date'].max()
    # We reserve the last (VALIDATION_STEPS or HORIZON) hours for validation targets
    cutoff_hours = max(VALIDATION_STEPS, FORECAST_HORIZON)
    val_target_start_date = max_train_date - pd.Timedelta(hours=cutoff_hours)

    # Train Mask: All data before the validation targets begin
    # (Actually, we usually train on everything available up to split point)
    mask_train = df_train['date'] < val_target_start_date

    # Val Mask: Must include LOOKBACK_WINDOW before the targets begin to form the X input
    val_input_start_date = val_target_start_date - pd.Timedelta(hours=LOOKBACK_WINDOW)
    mask_val = df_train['date'] >= val_input_start_date

    # We pass the dataframes directly to the generator
    train_gen = TrafficGenerator(
        df_train[mask_train].copy(),
        LOOKBACK_WINDOW,
        FORECAST_HORIZON,
        stride=STRIDE
    )

    val_gen = TrafficGenerator(
        df_train[mask_val].copy(),
        LOOKBACK_WINDOW,
        FORECAST_HORIZON,
        stride=1,
        shuffle=False
    )

    # CLEAR RAM: We don't need the raw train dataframe anymore
    del df_train
    gc.collect()

    # 3. Build Model
    model = Sequential()
    model.add(Input(shape=(LOOKBACK_WINDOW, 1)))
    model.add(LSTM(units=30, activation='tanh'))
    model.add(Dense(FORECAST_HORIZON))

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='mse')

    print("\n--- Training LSTM Model ---")
    start_time = time.time()

    history = model.fit(
        train_gen,
        epochs=3,
        validation_data=val_gen,
        verbose=1
    )

    if np.isnan(history.history['loss'][-1]):
        print("!!! CRITICAL FAILURE: NaN loss detected.")
        return pd.DataFrame()

    # 4. Evaluation Loop (Batch of 5 to save RAM)
    print("\n--- Predicting & Calculating Metrics (Batch Size = 5) ---")

    metrics_list = []
    BATCH_SIZE = 5
    total_samples = len(ids_list)

    for i in range(0, total_samples, BATCH_SIZE):
        # Process in chunks
        batch_ids = ids_list[i : i + BATCH_SIZE]
        X_batch = X_predict[i : i + BATCH_SIZE]

        # Predict Batch
        Y_log = model.predict(X_batch, verbose=0)
        Y_log = np.nan_to_num(Y_log, nan=0.0)
        Y_pred = np.expm1(Y_log)

        # Evaluate Batch
        for j, series_id in enumerate(batch_ids):
            # Prediction for this series
            y_p = Y_pred[j, :]

            # Ground Truth
            series_test_df = df_test[df_test['series_id'] == series_id].sort_values('date')
            y_t = series_test_df['sales'].values[:FORECAST_HORIZON]

            if len(y_t) < FORECAST_HORIZON:
                continue

            # Naive MAE (pre-calculated)
            mae_n = naive_mae_dict.get(series_id, 0.0)

            metrics = calculate_metrics(y_t, y_p, mae_n)
            metrics['unique_id'] = series_id
            metrics_list.append(metrics)

        # Clear Batch Memory
        del X_batch, Y_log, Y_pred
        gc.collect()

        if (i // BATCH_SIZE) % 10 == 0:
            print(f"   Processed {min(i + BATCH_SIZE, total_samples)}/{total_samples} series...", end='\r')

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    total_seconds = time.time() - start_time

    print("\n" + "="*35)
    print("FINAL TRAFFIC BENCHMARK RESULTS (LSTM - Hourly)")
    print("="*35)
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")
    print(f"Total Time: {total_seconds:.2f} seconds.")
    print("="*35)

    return final_metrics

if __name__ == '__main__':
    data = load_and_prepare_traffic_data()

    if data:
        df_train, df_test, X_predict, ids_list = data
        build_and_run_lstm(df_train, df_test, X_predict, ids_list)

        print("\n✅ LSTM Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# --- CONFIGURATION ---
TRAIN_FILE = 'ETTh1_Train.csv'
TEST_FILE = 'ETTh1_Test.csv'

# Standard Hourly Benchmark Settings
LOOKBACK_WINDOW = 96   # 4 Days context
FORECAST_HORIZON = 24  # 1 Day prediction
VALIDATION_HOURS = 168 # 1 Week validation

# ETTh1 is medium sized, so Stride 1 is fine (uses all data)
STRIDE = 1

np.random.seed(42)
tf.random.set_seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # ETTh1 can be negative (Temperature), so we don't clamp to 0 here.

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE (Standard version handles negative values via abs in denominator)
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE (Daily Seasonality = 24 hours)
    SEASONALITY = 24
    if len(y_train_hist) > SEASONALITY:
        naive_errors = np.abs(y_train_hist[SEASONALITY:] - y_train_hist[:-SEASONALITY])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}


# --- DATA PREPARATION ---

def create_sequences_and_normalize(train_df, test_df, window, horizon, stride=1):
    """
    Normalizes data (Z-Score) and creates sequences.
    Returns: X_train, Y_train, X_predict, ids_list, Scaler_Dict
    """
    X_train, Y_train = [], []
    X_predict_list = []
    ids_list = []

    # Store means and stds to inverse transform predictions later
    scaler_stats = {}

    # Identify all series
    series_ids = train_df['series_id'].unique()

    print(f"   Processing {len(series_ids)} series...")

    # Added tqdm for progress
    for series_id in tqdm(series_ids, desc="   Creating Sequences"):
        # 1. Get Train Data
        train_vals = train_df[train_df['series_id'] == series_id]['sales'].values.astype('float32')

        # 2. Calculate Stats (Z-Score Normalization)
        # We calculate stats ONLY on training data to avoid leakage
        mean = np.mean(train_vals)
        std = np.std(train_vals)
        if std == 0: std = 1e-5 # Prevent div by zero

        scaler_stats[series_id] = {'mean': mean, 'std': std}

        # 3. Normalize Train Data
        train_norm = (train_vals - mean) / std

        # 4. Create Train Sequences
        if len(train_norm) >= window + horizon:
            for i in range(0, len(train_norm) - window - horizon + 1, stride):
                X_train.append(train_norm[i:i + window])
                Y_train.append(train_norm[i + window: i + window + horizon])

        # 5. Prepare Prediction Input (Tail of Train) for Test
        # We need the last 'window' points of training to predict the first 'horizon' points of test
        if len(train_norm) >= window:
            X_predict_list.append(train_norm[-window:])
            ids_list.append(series_id)
        else:
            # Pad if short
            padded = np.pad(train_norm, (window - len(train_norm), 0), 'constant')
            X_predict_list.append(padded)
            ids_list.append(series_id)

    X_train = np.array(X_train, dtype='float32').reshape(-1, window, 1)
    Y_train = np.array(Y_train, dtype='float32')

    X_predict = np.array(X_predict_list, dtype='float32').reshape(-1, window, 1)

    return X_train, Y_train, X_predict, ids_list, scaler_stats

def load_and_prepare_etth1_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found.")
        return None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force string IDs
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Standardize column names
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Parse Dates
    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    # Sort
    df_train = df_train.sort_values(['series_id', 'date'])
    df_test = df_test.sort_values(['series_id', 'date'])

    # --- CREATE SEQUENCES ---
    print(f"2. Creating Sequences (Lookback={LOOKBACK_WINDOW}, Horizon={FORECAST_HORIZON})...")

    # Validation Split (Internal)
    # For ETTh1, we usually just take the last part of Train as Val for Early Stopping
    max_train_date = df_train['date'].max()
    val_start_date = max_train_date - pd.Timedelta(hours=VALIDATION_HOURS + FORECAST_HORIZON)

    train_subset = df_train[df_train['date'] < val_start_date].copy()
    val_subset = df_train[df_train['date'] >= val_start_date].copy()

    # Generate Train Sequences & Stats
    X_train, Y_train, _, _, scaler_stats = create_sequences_and_normalize(train_subset, None, LOOKBACK_WINDOW, FORECAST_HORIZON, stride=STRIDE)

    # Generate Val Sequences (using same stats)
    X_val, Y_val = [], []
    for series_id in val_subset['series_id'].unique():
        if series_id not in scaler_stats: continue

        stats = scaler_stats[series_id]
        vals = val_subset[val_subset['series_id'] == series_id]['sales'].values.astype('float32')
        norm = (vals - stats['mean']) / stats['std']

        if len(norm) >= LOOKBACK_WINDOW + FORECAST_HORIZON:
            for i in range(len(norm) - LOOKBACK_WINDOW - FORECAST_HORIZON + 1):
                X_val.append(norm[i:i+LOOKBACK_WINDOW])
                Y_val.append(norm[i+LOOKBACK_WINDOW:i+LOOKBACK_WINDOW+FORECAST_HORIZON])

    X_val = np.array(X_val, dtype='float32').reshape(-1, LOOKBACK_WINDOW, 1)
    Y_val = np.array(Y_val, dtype='float32')

    # Generate Prediction Input (X_predict) using Full Train and stats
    print("3. Preparing Prediction Inputs...")
    # We re-run the create function on the FULL train set to get the X_predict for the Test set
    # We discard the X_train/Y_train outputs here, we just want X_predict
    _, _, X_predict, ids_list, _ = create_sequences_and_normalize(df_train, None, LOOKBACK_WINDOW, FORECAST_HORIZON, stride=STRIDE)

    print(f"   Train Shape: {X_train.shape}")
    print(f"   Val Shape:   {X_val.shape}")

    return X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test


def build_and_run_lstm(X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test):

    # 1. Build Model
    model = Sequential()
    model.add(Input(shape=(LOOKBACK_WINDOW, 1)))
    model.add(LSTM(units=64, activation='tanh')) # 64 units for Hourly complexity
    model.add(Dense(FORECAST_HORIZON))

    # SAFETY: Clipvalue to prevent explosion
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='mse')

    print("\n--- Training LSTM Model ---")
    start_time = time.time()

    val_data = (X_val, Y_val) if len(X_val) > 0 else None

    history = model.fit(
        X_train, Y_train,
        epochs=5,
        batch_size=128,
        validation_data=val_data,
        verbose=1
    )

    if np.isnan(history.history['loss'][-1]):
        print("!!! CRITICAL FAILURE: NaN loss detected.")
        return pd.DataFrame()

    # 2. Prediction
    print("   Predicting...")
    # Prediction is in Z-Score scale
    Y_pred_norm = model.predict(X_predict)

    # 3. Evaluate Metrics (Inverse Transform per series)
    print("\n--- Calculating Metrics ---")
    metrics_list = []

    # Create dictionary for Train History (for MASE)
    history_dict = df_train.groupby('series_id')['sales'].apply(np.array).to_dict()

    for i, series_id in enumerate(ids_list):
        # Inverse Transform Prediction
        stats = scaler_stats[series_id]
        y_pred = (Y_pred_norm[i, :] * stats['std']) + stats['mean']

        # Get Ground Truth (First 24 hours of Test)
        series_test_df = df_test[df_test['series_id'] == series_id].sort_values('date')
        y_true = series_test_df['sales'].values[:FORECAST_HORIZON]

        if len(y_true) < FORECAST_HORIZON:
            continue

        y_hist = history_dict.get(series_id, np.array([]))

        metrics = calculate_metrics(y_true, y_pred, y_hist)
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    total_seconds = time.time() - start_time

    print("\n" + "="*35)
    print("FINAL ETTh1 RESULTS (LSTM)")
    print("="*35)
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")
    print(f"Total Time: {total_seconds:.2f} seconds.")
    print("="*35)

    return final_metrics

if __name__ == '__main__':
    data = load_and_prepare_etth1_data()

    if data:
        X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test = data
        build_and_run_lstm(X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test)

        print("\n✅ ETTh1 LSTM Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'ETTh1_Train.csv'
TEST_FILE = 'ETTh1_Test.csv'

# Validation size: Last 7 days (168 hours) of training data
VALIDATION_HOURS = 168

# Lag Shift: 1 means we use previous hour's data to predict current hour.
LAG_SHIFT = 1

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # ETTh1 values (Temperature/Load) can be negative if normalized.
    # If working with raw data (un-normalized), you might want to clamp.
    # Assuming raw data here based on processing script, but removing strict 0 clamp to be safe.
    # y_pred = np.maximum(0, y_pred)

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE (Daily Seasonality = 24 hours)
    SEASONALITY = 24
    if len(y_train_hist) > SEASONALITY:
        naive_errors = np.abs(y_train_hist[SEASONALITY:] - y_train_hist[:-SEASONALITY])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Forecast Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}

def generate_features(df):
    """Generates time-series features for Hourly ETTh1 data."""

    # --- 1. TIME FEATURES ---
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['hour'] = df['date'].dt.hour # CRITICAL FOR HOURLY DATA
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype('int64')

    # --- 2. CATEGORICAL FEATURES ---
    # Encode Series ID (HUFL, HULL, OT, etc.)
    encoder = LabelEncoder()
    df['series_id_encoded'] = encoder.fit_transform(df['series_id'].astype(str))

    # --- 3. LAG AND ROLLING WINDOW FEATURES ---
    # We define a helper to shift strictly within a Series ID
    def grouped_shift(col, shift_n):
        return df.groupby('series_id')[col].shift(shift_n)

    # Lags (Hourly Logic)
    df['lag_1']   = grouped_shift('sales', LAG_SHIFT)
    df['lag_6']   = grouped_shift('sales', LAG_SHIFT + 5)   # 6 hours ago
    df['lag_12']  = grouped_shift('sales', LAG_SHIFT + 11)  # 12 hours ago
    df['lag_24']  = grouped_shift('sales', LAG_SHIFT + 23)  # 1 day ago
    df['lag_168'] = grouped_shift('sales', LAG_SHIFT + 167) # 1 week ago

    # Rolling Statistics
    # Rolling Mean of last 24 hours (shifted to avoid leakage)
    df['rolling_mean_24'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(24).mean()
    )
    df['rolling_std_24'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(24).std()
    )

    # Rolling Mean of last 1 week
    df['rolling_mean_168'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(168).mean()
    )

    return df

def load_and_prepare_etth1_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found. Run the processing script first.")
        return None

    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force Series ID to string
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Rename 'value' -> 'sales' for internal consistency with metrics
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Mark splits before merging
    df_train['is_test'] = False
    df_test['is_test'] = True

    # Combine for Feature Engineering
    full_df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
    full_df = full_df.sort_values(['series_id', 'date'])

    print("2. Generating Features...")
    full_df = generate_features(full_df)

    # --- SPLITTING ---
    X_test_full = full_df[full_df['is_test'] == True].copy()
    train_full = full_df[full_df['is_test'] == False].copy()

    # Create Validation Set (Last 168 hours of Train)
    max_train_date = train_full['date'].max()
    val_start_date = max_train_date - pd.Timedelta(hours=VALIDATION_HOURS - 1)

    X_train_df = train_full[train_full['date'] < val_start_date]
    X_val_df = train_full[train_full['date'] >= val_start_date]

    # Define Feature Columns
    drop_cols = ['sales', 'date', 'series_id', 'is_test', 'split_type']
    features = [c for c in full_df.columns if c not in drop_cols]

    print(f"   Features used: {features}")

    # Create Final Arrays
    X_train = X_train_df[features]
    y_train = X_train_df['sales']

    X_val = X_val_df[features]
    y_val = X_val_df['sales']

    X_test = X_test_full[features]
    y_test = X_test_full['sales']

    # Clean NaNs caused by lags
    valid_indices = X_train.dropna().index
    X_train = X_train.loc[valid_indices]
    y_train = y_train.loc[valid_indices]

    print("\n--- Final Data Shapes ---")
    print(f"Train Set: {len(X_train)} rows")
    print(f"Val Set:   {len(X_val)} rows")
    print(f"Test Set:  {len(X_test)} rows")

    return X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full

def train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full_history):

    print("\n--- Training XGBoost Model ---")

    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': 0.05,
        'max_depth': 8,
        'seed': 42,
        'nthread': -1,
        'tree_method': 'hist'
    }
    num_round = 1000

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)

    eval_list = [(dtrain, 'train'), (dval, 'validation')]

    bst = xgb.train(
        xgb_params,
        dtrain,
        num_round,
        evals=eval_list,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    print(f"   Best Iteration: {bst.best_iteration}")

    # Prediction
    y_pred_test = bst.predict(dtest, iteration_range=(0, bst.best_iteration))

    # --- METRICS CALCULATION ---
    print("\n--- Calculating Final Benchmarks ---")

    results_df = X_test_full.copy()
    results_df['y_pred'] = y_pred_test
    results_df['sales'] = y_test

    metrics_list = []
    unique_series = results_df['series_id'].unique()

    for i, series_id in enumerate(unique_series):
        series_results = results_df[results_df['series_id'] == series_id]

        history_series = train_full_history[train_full_history['series_id'] == series_id]['sales'].values

        metrics = calculate_metrics(
            series_results['sales'].values,
            series_results['y_pred'].values,
            history_series
        )
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == '__main__':
    data_tuple = load_and_prepare_etth1_data()

    if data_tuple:
        X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full = data_tuple
        train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full)

        print("\n✅ ETTh1 XGBoost Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'ETTh1_Train.csv'
TEST_FILE = 'ETTh1_Test.csv'

# Seasonality for Hourly Data (ETTh1)
# Primary cycle is Daily (24 hours).
# SNaive will repeat the last 24 hours of training into the future.
SEASONALITY = 24

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # ETTh1 values can be negative (normalized), so we don't strict clamp to 0.
    # y_pred = np.maximum(0, y_pred)

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # Calculate seasonal naive MAE on the entire training history
    if len(y_train_hist) > SEASONALITY:
        naive_errors = np.abs(y_train_hist[SEASONALITY:] - y_train_hist[:-SEASONALITY])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Forecast Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}


def load_etth1_data():
    """Loads the pre-split ETTh1 CSVs."""
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"Error: {TRAIN_FILE} or {TEST_FILE} not found.")
        return None, None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force series_id to string
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Rename 'value' -> 'sales' (internal standard)
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Ensure dates are datetime objects
    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    print(f"   Train Rows: {len(df_train)}")
    print(f"   Test Rows:  {len(df_test)}")

    return df_train, df_test


def run_snaive_benchmark(df_train, df_test):
    """
    Runs Seasonal Naive: Forecast = Last 24 hours of training, repeated.
    """
    metrics_list = []

    # Group by Series ID for fast access
    train_groups = df_train.groupby('series_id')
    test_groups = df_test.groupby('series_id')

    unique_ids = df_test['series_id'].unique()

    print(f"\n2. Running Seasonal Naive Benchmark (S={SEASONALITY}) on {len(unique_ids)} series...")
    start_time = time.time()

    for i, series_id in enumerate(unique_ids):
        try:
            # Get Training History (sorted by date)
            train_series = train_groups.get_group(series_id).sort_values('date')
            y_train = train_series['sales'].values

            # Get Test Truth (sorted by date)
            test_series = test_groups.get_group(series_id).sort_values('date')
            y_test = test_series['sales'].values

            # Prediction Horizon needed
            horizon = len(y_test)

            if len(y_train) < SEASONALITY:
                # Fallback if history is too short (rare for ETTh1)
                y_pred = np.tile(y_train[-1:], horizon)
            else:
                # SNaive Logic: Grab last 24 hours (1 Day)
                last_season = y_train[-SEASONALITY:]

                # Repeat (Tile) this pattern to fill the horizon
                repetitions = int(np.ceil(horizon / SEASONALITY))
                y_pred = np.tile(last_season, repetitions)[:horizon]

            # Calculate Metrics
            metrics = calculate_metrics(y_test, y_pred, y_train)
            metrics['unique_id'] = series_id
            metrics_list.append(metrics)

        except KeyError:
            continue

    end_time = time.time()
    total_time = end_time - start_time

    # Aggregate
    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"\n\n--- Final SNaive Results ({total_time:.4f}s) ---")
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == "__main__":
    df_train, df_test = load_etth1_data()

    if df_train is not None:
        run_snaive_benchmark(df_train, df_test)
        print("\n✅ ETTh1 SNaive Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import mean_squared_error
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'Exchange_Rate_Train.csv'
TEST_FILE = 'Exchange_Rate_Test.csv'

# Exchange Rate is DAILY data.
# While often modeled as a Random Walk (S=1), a standard SNaive benchmark
# for daily data uses Weekly seasonality (S=7) to capture day-of-week effects.
SEASONALITY = 7

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # Calculate seasonal naive MAE on the entire training history
    if len(y_train_hist) > SEASONALITY:
        naive_errors = np.abs(y_train_hist[SEASONALITY:] - y_train_hist[:-SEASONALITY])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Forecast Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}


def load_exchange_data():
    """Loads the pre-split Exchange Rate CSVs."""
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print(f"Error: {TRAIN_FILE} or {TEST_FILE} not found.")
        return None, None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force series_id to string
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Rename 'value' -> 'sales' (internal standard)
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Ensure dates are datetime objects
    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    print(f"   Train Rows: {len(df_train)}")
    print(f"   Test Rows:  {len(df_test)}")

    return df_train, df_test


def run_snaive_benchmark(df_train, df_test):
    """
    Runs Seasonal Naive: Forecast = Last 7 days of training, repeated.
    """
    metrics_list = []

    # Group by Series ID for fast access
    train_groups = df_train.groupby('series_id')
    test_groups = df_test.groupby('series_id')

    unique_ids = df_test['series_id'].unique()

    print(f"\n2. Running Seasonal Naive Benchmark (S={SEASONALITY}) on {len(unique_ids)} series...")
    start_time = time.time()

    for i, series_id in enumerate(unique_ids):
        try:
            # Get Training History (sorted by date)
            train_series = train_groups.get_group(series_id).sort_values('date')
            y_train = train_series['sales'].values

            # Get Test Truth (sorted by date)
            test_series = test_groups.get_group(series_id).sort_values('date')
            y_test = test_series['sales'].values

            # Prediction Horizon needed
            horizon = len(y_test)

            if len(y_train) < SEASONALITY:
                # Fallback if history is too short
                y_pred = np.tile(y_train[-1:], horizon)
            else:
                # SNaive Logic: Grab last 7 days (Weekly Seasonality)
                last_season = y_train[-SEASONALITY:]

                # Repeat (Tile) this pattern to fill the horizon
                repetitions = int(np.ceil(horizon / SEASONALITY))
                y_pred = np.tile(last_season, repetitions)[:horizon]

            # Calculate Metrics
            metrics = calculate_metrics(y_test, y_pred, y_train)
            metrics['unique_id'] = series_id
            metrics_list.append(metrics)

        except KeyError:
            continue

    end_time = time.time()
    total_time = end_time - start_time

    # Aggregate
    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"\n\n--- Final SNaive Results ({total_time:.4f}s) ---")
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == "__main__":
    df_train, df_test = load_exchange_data()

    if df_train is not None:
        run_snaive_benchmark(df_train, df_test)
        print("\n✅ Exchange Rate SNaive Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import time
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'Exchange_Rate_Train.csv'
TEST_FILE = 'Exchange_Rate_Test.csv'

# Forecast Horizon: 96 Days (Standard Long-Term Forecast for Exchange Rate)
FORECAST_HORIZON = 96
VALIDATION_DAYS = 96

# Lag Shift: 1 means we use yesterday's data to predict today.
LAG_SHIFT = 1

# Set random seed
np.random.seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # Exchange rates are strictly positive.
    y_pred = np.maximum(1e-5, y_pred)

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # Exchange rate data typically doesn't have strong weekly seasonality like retail.
    # It behaves more like a random walk. We use naive lag-1 for MASE denominator.
    mae_naive = np.mean(np.abs(np.diff(y_train_hist)))

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Forecast Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}

def generate_features(df):
    """Generates time-series features for Daily Exchange Rate data."""

    # --- 1. TIME FEATURES ---
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday
    df['weekofyear'] = df['date'].dt.isocalendar().week.astype('int64')

    # --- 2. CATEGORICAL FEATURES ---
    # Encode Series ID (Country/Currency)
    encoder = LabelEncoder()
    df['series_id_encoded'] = encoder.fit_transform(df['series_id'].astype(str))

    # --- 3. LAG AND ROLLING WINDOW FEATURES ---
    # We define a helper to shift strictly within a Series ID
    def grouped_shift(col, shift_n):
        return df.groupby('series_id')[col].shift(shift_n)

    # Lags (Daily Logic)
    df['lag_1']  = grouped_shift('sales', LAG_SHIFT)
    df['lag_7']  = grouped_shift('sales', LAG_SHIFT + 6)
    df['lag_14'] = grouped_shift('sales', LAG_SHIFT + 13)
    df['lag_30'] = grouped_shift('sales', LAG_SHIFT + 29)
    df['lag_96'] = grouped_shift('sales', LAG_SHIFT + 95) # Horizon lag

    # Rolling Statistics
    # Rolling Mean of last 7/30 days (shifted to avoid leakage)
    df['rolling_mean_7'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(7).mean()
    )
    df['rolling_std_7'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(7).std()
    )

    df['rolling_mean_30'] = df.groupby('series_id')['sales'].transform(
        lambda x: x.shift(LAG_SHIFT).rolling(30).mean()
    )

    return df

def load_and_prepare_exchange_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found. Run the previous split script first.")
        return None

    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force Series ID to string
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Rename 'value' -> 'sales' (internal standard naming)
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Mark splits before merging
    df_train['is_test'] = False
    df_test['is_test'] = True

    # Combine for Feature Engineering (Preserve history at boundaries)
    full_df = pd.concat([df_train, df_test], axis=0, ignore_index=True)
    full_df = full_df.sort_values(['series_id', 'date'])

    print("2. Generating Features...")
    full_df = generate_features(full_df)

    # --- SPLITTING ---
    # 1. Recover Test Set
    X_test_full = full_df[full_df['is_test'] == True].copy()

    # 2. Recover Train Set (All non-test data)
    train_full = full_df[full_df['is_test'] == False].copy()

    # 3. Create Validation Set from the END of the Train Set
    max_train_date = train_full['date'].max()
    val_start_date = max_train_date - pd.Timedelta(days=VALIDATION_DAYS - 1)

    # Split Train/Val
    X_train_df = train_full[train_full['date'] < val_start_date]
    X_val_df = train_full[train_full['date'] >= val_start_date]

    # Define Feature Columns (Drop non-feature cols)
    drop_cols = ['sales', 'date', 'series_id', 'is_test', 'split_type']
    features = [c for c in full_df.columns if c not in drop_cols]

    print(f"   Features used: {features}")

    # Create Final Arrays
    X_train = X_train_df[features]
    y_train = X_train_df['sales']

    X_val = X_val_df[features]
    y_val = X_val_df['sales']

    X_test = X_test_full[features]
    y_test = X_test_full['sales']

    # Clean NaNs caused by lags (mostly in early train data)
    valid_indices = X_train.dropna().index
    X_train = X_train.loc[valid_indices]
    y_train = y_train.loc[valid_indices]

    print("\n--- Final Data Shapes ---")
    print(f"Train Set: {len(X_train)} rows")
    print(f"Val Set:   {len(X_val)} rows")
    print(f"Test Set:  {len(X_test)} rows")

    return X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full

def train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full_history):

    print("\n--- Training XGBoost Model ---")

    xgb_params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': 0.05,
        'max_depth': 6, # Slightly shallower for exchange rates (less noisy than retail)
        'seed': 42,
        'nthread': -1,
        'tree_method': 'hist'
    }
    num_round = 1000

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)

    eval_list = [(dtrain, 'train'), (dval, 'validation')]

    bst = xgb.train(
        xgb_params,
        dtrain,
        num_round,
        evals=eval_list,
        early_stopping_rounds=50,
        verbose_eval=False
    )

    print(f"   Best Iteration: {bst.best_iteration}")

    # Prediction
    y_pred_test = bst.predict(dtest, iteration_range=(0, bst.best_iteration))

    # --- METRICS CALCULATION ---
    print("\n--- Calculating Final Benchmarks ---")

    results_df = X_test_full.copy()
    results_df['y_pred'] = y_pred_test
    # Ensure alignment
    results_df['sales'] = y_test

    metrics_list = []

    unique_series = results_df['series_id'].unique()

    for i, series_id in enumerate(unique_series):
        series_results = results_df[results_df['series_id'] == series_id]

        # Use full training history (Train + Val) for MASE denominator
        history_series = train_full_history[train_full_history['series_id'] == series_id]['sales'].values

        metrics = calculate_metrics(
            series_results['sales'].values,
            series_results['y_pred'].values,
            history_series
        )
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")

    return final_metrics

if __name__ == '__main__':
    # Load data
    data_tuple = load_and_prepare_exchange_data()

    if data_tuple:
        X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full = data_tuple

        # Train and Eval
        train_and_evaluate_xgboost(X_train, y_train, X_val, y_val, X_test, y_test, X_test_full, train_full)

        print("\n✅ Exchange Rate XGBoost Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import time
import os
import gc
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# --- CONFIGURATION ---
TRAIN_FILE = 'Exchange_Rate_Train.csv'
TEST_FILE = 'Exchange_Rate_Test.csv'

# Standard Exchange Rate Benchmark Settings
LOOKBACK_WINDOW = 96    # 96 Days context
FORECAST_HORIZON = 96   # 96 Days prediction
VALIDATION_DAYS = 96    # 96 Days validation

# Exchange Rate data is small (~8 series, ~7k rows), so Stride 1 is fine
STRIDE = 1

np.random.seed(42)
tf.random.set_seed(42)

# --- METRIC FUNCTIONS ---

def calculate_metrics(y_true, y_pred, y_train_hist):
    """Calculates MAE, MSE, RMSE, sMAPE, MASE, and Bias."""

    # Exchange rates are strictly positive
    y_pred = np.maximum(0, y_pred)

    # 1. MAE (Mean Absolute Error)
    mae_forecast = np.mean(np.abs(y_true - y_pred))

    # 2. MSE (Mean Squared Error)
    mse = mean_squared_error(y_true, y_pred)

    # 3. RMSE
    rmse = np.sqrt(mse)

    # 4. sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    mask = denominator != 0
    smape = np.mean(200 * np.abs(y_pred[mask] - y_true[mask]) / denominator[mask]) if np.sum(mask) > 0 else 0.0

    # 5. MASE
    # Exchange Rate is Daily. Often modeled as Random Walk (Naive 1).
    # We use lag=1 for the denominator to be consistent with financial baselines.
    # (If using SNaive with weekly seasonality, one might use 7, but 1 is standard for MASE here).
    seasonality = 1
    if len(y_train_hist) > seasonality:
        naive_errors = np.abs(y_train_hist[seasonality:] - y_train_hist[:-seasonality])
        mae_naive = np.mean(naive_errors)
    elif len(y_train_hist) > 1:
        mae_naive = np.mean(np.abs(np.diff(y_train_hist)))
    else:
        mae_naive = 0.0

    mase = mae_forecast / mae_naive if mae_naive > 1e-9 else np.nan

    # 6. Bias
    sum_actual = np.sum(y_true)
    bias = (np.sum(y_pred) - sum_actual) / sum_actual * 100 if sum_actual != 0 else np.nan

    return {'MAE': mae_forecast, 'MSE': mse, 'RMSE': rmse, 'sMAPE': smape, 'MASE': mase, 'Bias': bias}


# --- DATA PREPARATION ---

def create_sequences_and_normalize(train_df, test_df, window, horizon, stride=1):
    """
    Normalizes data (Z-Score) and creates sequences.
    Returns: X_train, Y_train, X_predict, ids_list, Scaler_Dict
    """
    X_train, Y_train = [], []
    X_predict_list = []
    ids_list = []

    # Store means and stds to inverse transform predictions later
    scaler_stats = {}

    # Identify all series
    series_ids = train_df['series_id'].unique()

    print(f"   Processing {len(series_ids)} series...")

    for series_id in tqdm(series_ids, desc="   Creating Sequences"):
        # 1. Get Train Data
        train_vals = train_df[train_df['series_id'] == series_id]['sales'].values.astype('float32')

        # 2. Calculate Stats (Z-Score Normalization)
        # We calculate stats ONLY on training data to avoid leakage
        mean = np.mean(train_vals)
        std = np.std(train_vals)
        if std == 0: std = 1e-5 # Prevent div by zero

        scaler_stats[series_id] = {'mean': mean, 'std': std}

        # 3. Normalize Train Data
        train_norm = (train_vals - mean) / std

        # 4. Create Train Sequences
        if len(train_norm) >= window + horizon:
            for i in range(0, len(train_norm) - window - horizon + 1, stride):
                X_train.append(train_norm[i:i + window])
                Y_train.append(train_norm[i + window: i + window + horizon])

        # 5. Prepare Prediction Input (Tail of Train) for Test
        # We need the last 'window' points of training to predict the first 'horizon' points of test
        if len(train_norm) >= window:
            X_predict_list.append(train_norm[-window:])
            ids_list.append(series_id)
        else:
            # Pad if short
            padded = np.pad(train_norm, (window - len(train_norm), 0), 'constant')
            X_predict_list.append(padded)
            ids_list.append(series_id)

    X_train = np.array(X_train, dtype='float32').reshape(-1, window, 1)
    Y_train = np.array(Y_train, dtype='float32')

    X_predict = np.array(X_predict_list, dtype='float32').reshape(-1, window, 1)

    return X_train, Y_train, X_predict, ids_list, scaler_stats

def load_and_prepare_exchange_data():
    print("1. Loading Data Splits...")

    if not os.path.exists(TRAIN_FILE) or not os.path.exists(TEST_FILE):
        print("Error: Train/Test files not found.")
        return None

    # Load data
    df_train = pd.read_csv(TRAIN_FILE, low_memory=False)
    df_test = pd.read_csv(TEST_FILE, low_memory=False)

    # Force string IDs
    df_train['series_id'] = df_train['series_id'].astype(str)
    df_test['series_id'] = df_test['series_id'].astype(str)

    # Standardize column names
    if 'value' in df_train.columns:
        df_train.rename(columns={'value': 'sales'}, inplace=True)
        df_test.rename(columns={'value': 'sales'}, inplace=True)

    # Parse Dates
    df_train['date'] = pd.to_datetime(df_train['date'])
    df_test['date'] = pd.to_datetime(df_test['date'])

    # Sort
    df_train = df_train.sort_values(['series_id', 'date'])
    df_test = df_test.sort_values(['series_id', 'date'])

    # --- CREATE SEQUENCES ---
    print(f"2. Creating Sequences (Lookback={LOOKBACK_WINDOW}, Horizon={FORECAST_HORIZON})...")

    # Validation Split (Internal)
    # Take the last part of Train as Val for Early Stopping
    max_train_date = df_train['date'].max()
    val_start_date = max_train_date - pd.Timedelta(days=VALIDATION_DAYS + FORECAST_HORIZON)

    train_subset = df_train[df_train['date'] < val_start_date].copy()
    val_subset = df_train[df_train['date'] >= val_start_date].copy()

    # Generate Train Sequences & Stats
    X_train, Y_train, _, _, scaler_stats = create_sequences_and_normalize(train_subset, None, LOOKBACK_WINDOW, FORECAST_HORIZON, stride=STRIDE)

    # Generate Val Sequences (using same stats)
    X_val, Y_val = [], []
    for series_id in val_subset['series_id'].unique():
        if series_id not in scaler_stats: continue

        stats = scaler_stats[series_id]
        vals = val_subset[val_subset['series_id'] == series_id]['sales'].values.astype('float32')
        norm = (vals - stats['mean']) / stats['std']

        if len(norm) >= LOOKBACK_WINDOW + FORECAST_HORIZON:
            for i in range(len(norm) - LOOKBACK_WINDOW - FORECAST_HORIZON + 1):
                X_val.append(norm[i:i+LOOKBACK_WINDOW])
                Y_val.append(norm[i+LOOKBACK_WINDOW:i+LOOKBACK_WINDOW+FORECAST_HORIZON])

    X_val = np.array(X_val, dtype='float32').reshape(-1, LOOKBACK_WINDOW, 1)
    Y_val = np.array(Y_val, dtype='float32')

    # Generate Prediction Input (X_predict) using Full Train and stats
    print("3. Preparing Prediction Inputs...")
    _, _, X_predict, ids_list, _ = create_sequences_and_normalize(df_train, None, LOOKBACK_WINDOW, FORECAST_HORIZON, stride=STRIDE)

    print(f"   Train Shape: {X_train.shape}")
    print(f"   Val Shape:   {X_val.shape}")

    return X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test


def build_and_run_lstm(X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test):

    # 1. Build Model
    model = Sequential()
    model.add(Input(shape=(LOOKBACK_WINDOW, 1)))
    model.add(LSTM(units=64, activation='tanh'))
    model.add(Dense(FORECAST_HORIZON))

    # SAFETY: Clipvalue to prevent explosion
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, clipvalue=0.5)
    model.compile(optimizer=optimizer, loss='mse')

    print("\n--- Training LSTM Model ---")
    start_time = time.time()

    val_data = (X_val, Y_val) if len(X_val) > 0 else None

    history = model.fit(
        X_train, Y_train,
        epochs=10, # More epochs for smaller dataset
        batch_size=64,
        validation_data=val_data,
        verbose=1
    )

    if np.isnan(history.history['loss'][-1]):
        print("!!! CRITICAL FAILURE: NaN loss detected.")
        return pd.DataFrame()

    # 2. Prediction
    print("   Predicting...")
    # Prediction is in Z-Score scale
    Y_pred_norm = model.predict(X_predict)

    # 3. Evaluate Metrics (Inverse Transform per series)
    print("\n--- Calculating Metrics ---")
    metrics_list = []

    # Create dictionary for Train History (for MASE)
    history_dict = df_train.groupby('series_id')['sales'].apply(np.array).to_dict()

    for i, series_id in enumerate(ids_list):
        # Inverse Transform Prediction
        stats = scaler_stats[series_id]
        y_pred = (Y_pred_norm[i, :] * stats['std']) + stats['mean']

        # Get Ground Truth (First 96 days of Test)
        series_test_df = df_test[df_test['series_id'] == series_id].sort_values('date')
        y_true = series_test_df['sales'].values[:FORECAST_HORIZON]

        if len(y_true) < FORECAST_HORIZON:
            continue

        y_hist = history_dict.get(series_id, np.array([]))

        metrics = calculate_metrics(y_true, y_pred, y_hist)
        metrics['unique_id'] = series_id
        metrics_list.append(metrics)

    final_metrics = pd.DataFrame(metrics_list).dropna(subset=['MASE'])

    total_seconds = time.time() - start_time

    print("\n" + "="*35)
    print("FINAL EXCHANGE RATE RESULTS (LSTM)")
    print("="*35)
    print(f"Evaluated Series: {len(final_metrics)}")
    print(f"Mean MAE:   {final_metrics['MAE'].mean():.4f}")
    print(f"Mean MSE:   {final_metrics['MSE'].mean():.4f}")
    print(f"Mean RMSE:  {final_metrics['RMSE'].mean():.4f}")
    print(f"Mean sMAPE: {final_metrics['sMAPE'].mean():.4f}%")
    print(f"Mean MASE:  {final_metrics['MASE'].mean():.4f}")
    print(f"Mean Bias:  {final_metrics['Bias'].mean():.4f}%")
    print(f"Total Time: {total_seconds:.2f} seconds.")
    print("="*35)

    return final_metrics

if __name__ == '__main__':
    data = load_and_prepare_exchange_data()

    if data:
        X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test = data
        build_and_run_lstm(X_train, Y_train, X_val, Y_val, X_predict, ids_list, scaler_stats, df_train, df_test)

        print("\n✅ Exchange Rate LSTM Benchmark Complete.")

In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
TRAIN_FILE = 'Daily-train.csv'
TEST_FILE = 'Daily-test.csv'
INFO_FILE = 'M4-info.csv'

def process_m4_data_fixed():
    print("1. Loading Data...")
    df_train = pd.read_csv(TRAIN_FILE)
    df_test = pd.read_csv(TEST_FILE)
    df_info = pd.read_csv(INFO_FILE)

    # Standardize ID columns
    df_train.rename(columns={'V1': 'series_id'}, inplace=True)
    df_test.rename(columns={'V1': 'series_id'}, inplace=True)
    df_info.rename(columns={'M4id': 'series_id'}, inplace=True)

    print(f"   Train Shape: {df_train.shape}")
    print(f"   Test Shape: {df_test.shape}")

    # --- STEP 2: ROBUST MELT (Skip complex metadata logic) ---
    print("2. Melting Training Data...")

    # Identify value columns (V2, V3...)
    train_value_cols = [c for c in df_train.columns if c.startswith('V')]

    # Melt
    df_train_long = df_train.melt(
        id_vars=['series_id'],
        value_vars=train_value_cols,
        var_name='col_code',
        value_name='sales'
    )

    # Drop NaNs (This removes the empty tail of the wide matrix)
    df_train_long.dropna(subset=['sales'], inplace=True)

    # Calculate Day Offset (V2 -> 0, V3 -> 1...)
    # Extracts digits from 'V2' -> 2, subtracts 2 -> 0
    df_train_long['day_offset'] = df_train_long['col_code'].str.extract('(\d+)').astype(int) - 2

    # Join with Start Date from Info file
    print("   Mapping Dates...")
    df_train_final = df_train_long.merge(df_info[['series_id', 'StartingDate']], on='series_id', how='left')
    df_train_final['start_date'] = pd.to_datetime(df_train_final['StartingDate'])

    # Calculate Actual Date
    df_train_final['date'] = df_train_final['start_date'] + pd.to_timedelta(df_train_final['day_offset'], unit='D')

    # Final Cleanup
    df_train_output = df_train_final[['series_id', 'date', 'sales']].sort_values(['series_id', 'date'])
    df_train_output['split_type'] = 'TRAIN'

    print(f"   Saving M4_Daily_Train_PLX.csv ({len(df_train_output)} rows)...")
    df_train_output.to_csv('M4_Daily_Train_PLX.csv', index=False)


    # --- STEP 3: PROCESS TEST DATA ---
    print("3. Melting Test Data...")

    # Identify Test Value Columns
    test_value_cols = [c for c in df_test.columns if c.startswith('V')]

    df_test_long = df_test.melt(
        id_vars=['series_id'],
        value_vars=test_value_cols,
        var_name='col_code',
        value_name='sales'
    )
    df_test_long.dropna(subset=['sales'], inplace=True)

    # Calculate Test Day Offset
    # V2 in Test file is the 1st day of forecast.
    # We need to find the END of the training data to know where Test starts.

    # Get the max date per series from the training set we just built
    train_end_dates = df_train_output.groupby('series_id')['date'].max().reset_index()
    train_end_dates.rename(columns={'date': 'train_end_date'}, inplace=True)

    # Merge this end date into the test set
    df_test_final = df_test_long.merge(train_end_dates, on='series_id', how='left')

    # Extract test offset (V2 -> 0, V3 -> 1...)
    df_test_final['test_step'] = df_test_final['col_code'].str.extract('(\d+)').astype(int) - 2

    # Calculate Actual Test Date: Train End + 1 Day + Step Offset
    df_test_final['date'] = df_test_final['train_end_date'] + pd.to_timedelta(df_test_final['test_step'] + 1, unit='D')

    # Final Cleanup
    df_test_output = df_test_final[['series_id', 'date', 'sales']].sort_values(['series_id', 'date'])
    df_test_output['split_type'] = 'TEST'

    print(f"   Saving M4_Daily_Test_PLX.csv ({len(df_test_output)} rows)...")
    df_test_output.to_csv('M4_Daily_Test_PLX.csv', index=False)

    print("\n✅ SUCCESS! M4 Data Reprocessed correctly.")
    print(f"Train Row Count: {len(df_train_output)}")
    print(f"Test Row Count:  {len(df_test_output)}")

if __name__ == "__main__":
    process_m4_data_fixed()

In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
INPUT_FILE = 'traffic.csv'
TRAIN_OUTPUT = 'Traffic_Daily_Train.csv'
TEST_OUTPUT = 'Traffic_Daily_Test.csv'

def process_traffic_data():
    print("1. Loading Data...")
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found.")
        return

    # Load data
    # Assuming standard traffic.csv where col 0 is date and rest are sensors
    df = pd.read_csv(INPUT_FILE)

    # Ensure date is actually datetime objects
    # Adjust 'date' below if your timestamp column has a different name
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    else:
        # Fallback if no header or different name, assuming first column is date
        print("   Note: 'date' column not found, assuming first column is timestamp.")
        date_col_name = df.columns[0]
        df.rename(columns={date_col_name: 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'])

    print(f"   Total Data Shape: {df.shape}")

    # --- STEP 2: SPLIT DATA (TIME BASED) ---
    print("2. Splitting Train/Test (Last 20% for Test)...")

    # Calculate split index based on time (rows)
    total_rows = len(df)
    split_index = int(total_rows * 0.8) # 80% cutoff

    # Slice the dataframe chronologically
    df_train_wide = df.iloc[:split_index].copy()
    df_test_wide = df.iloc[split_index:].copy()

    print(f"   Train Rows (Wide): {len(df_train_wide)}")
    print(f"   Test Rows (Wide):  {len(df_test_wide)}")

    # --- STEP 3: MELT AND PROCESS ---

    def process_split(df_wide, split_name):
        print(f"   Processing {split_name} split...")

        # Identify sensor columns (all columns except 'date')
        sensor_cols = [c for c in df_wide.columns if c != 'date']

        # Melt from Wide (Date, Sensor1, Sensor2...) to Long (Date, Series_ID, Value)
        df_long = df_wide.melt(
            id_vars=['date'],
            value_vars=sensor_cols,
            var_name='series_id',
            value_name='value'
        )

        # Sort for cleanliness
        df_long = df_long.sort_values(['series_id', 'date'])

        # Add metadata
        df_long['split_type'] = split_name

        return df_long

    # Process Train
    df_train_final = process_split(df_train_wide, 'TRAIN')

    # Process Test
    df_test_final = process_split(df_test_wide, 'TEST')

    # --- STEP 4: SAVING ---
    print(f"3. Saving Output Files...")

    print(f"   Saving {TRAIN_OUTPUT} ({len(df_train_final)} rows)...")
    df_train_final.to_csv(TRAIN_OUTPUT, index=False)

    print(f"   Saving {TEST_OUTPUT} ({len(df_test_final)} rows)...")
    df_test_final.to_csv(TEST_OUTPUT, index=False)

    print("\n✅ SUCCESS! Traffic Data Reprocessed correctly.")

if __name__ == "__main__":
    process_traffic_data()
    
    TRAIN_OUTPUT = 'Traffic_Daily_Train.csv'
TEST_OUTPUT = 'Traffic_Daily_Test.csv'

df_train = pd.read_csv(TRAIN_OUTPUT)
df_test = pd.read_csv(TEST_OUTPUT)

df_train.head()
# df_test.head()

In [ ]:
import pandas as pd
import numpy as np
import time
import os

# --- Configuration ---
INPUT_SALES_FILE = 'sales_train_evaluation.csv'
OUTPUT_SALES_FILE = 'sales_train_sample.csv' # Target file for upload
M5_START_DATE = '2011-01-29'
CHUNK_SIZE = 5000 # Read 5000 series (rows) at a time to find IDs
SAMPLE_SIZE = 500 # The target sample size for the benchmark

def transform_m5_sales_sample():
    print(f"Starting transformation and sampling of {INPUT_SALES_FILE}...")
    start_time = time.time()
    total_output_rows = 0

    # 1. --- PASS 1: IDENTIFY SAMPLE IDs (Low RAM usage) ---
    print("PASS 1: Identifying 500 random series IDs (using random seed 42)...")

    try:
        # Read only the 'id' column to get the complete list of IDs
        id_df = pd.read_csv(INPUT_SALES_FILE, usecols=['id'])
    except FileNotFoundError:
        print(f"ERROR: {INPUT_SALES_FILE} not found. Please ensure the file is accessible.")
        return

    all_series_ids = id_df['id'].unique()

    # Select the random sample (500 IDs)
    if len(all_series_ids) > SAMPLE_SIZE:
        np.random.seed(42) # Set seed for reproducible science
        sampled_ids = np.random.choice(all_series_ids, size=SAMPLE_SIZE, replace=False)
    else:
        sampled_ids = all_series_ids

    print(f"Successfully selected {len(sampled_ids)} IDs.")

    # 2. --- PASS 2: MELT AND FILTER (Chunking/Low RAM usage) ---
    print("PASS 2: Melting and filtering only sampled IDs...")

    # Clean output file
    if os.path.exists(OUTPUT_SALES_FILE):
        os.remove(OUTPUT_SALES_FILE)

    is_first_chunk = True

    # Read the full file in chunks (including all columns this time)
    reader = pd.read_csv(INPUT_SALES_FILE, chunksize=CHUNK_SIZE)

    for i, chunk in enumerate(reader):
        # --- A. FILTER THE CHUNK ---
        # Only keep rows whose 'id' is in our sampled list
        chunk_filtered = chunk[chunk['id'].isin(sampled_ids)].copy()

        if chunk_filtered.empty:
            continue # Skip chunks that contain none of our target IDs

        # --- B. MELT (Wide to Long Transformation) ---
        day_cols = [c for c in chunk_filtered.columns if c.startswith('d_')]
        id_cols = [c for c in chunk_filtered.columns if c not in day_cols]

        m5_df_long = chunk_filtered.melt(
            id_vars=id_cols,
            value_vars=day_cols,
            var_name='day_index',
            value_name='sales_value'
        ).dropna(subset=['sales_value'])

        # --- C. GENERATE DATES ---
        m5_df_long['day_num'] = m5_df_long['day_index'].str.replace('d_', '').astype(int)
        m5_df_long['date'] = pd.to_datetime(M5_START_DATE) + pd.to_timedelta(m5_df_long['day_num'] - 1, unit='D')

        # 3. Finalize Structure
        final_df = m5_df_long.rename(columns={'id': 'series_id', 'sales_value': 'sales'})

        # Select the required columns for the TimesFM benchmark and XGBoost features:
        final_df = final_df[['series_id', 'date', 'sales', 'item_id', 'store_id', 'dept_id', 'cat_id', 'state_id']]

        # 4. Write to CSV (Append mode after the first chunk)
        header = is_first_chunk
        mode = 'w' if is_first_chunk else 'a'

        final_df.to_csv(OUTPUT_SALES_FILE, mode=mode, header=header, index=False)

        total_output_rows += len(final_df)
        is_first_chunk = False

        print(f"-> Chunk {i+1} processed. Total rows written: {total_output_rows}")

    end_time = time.time()

    print("\n" + "="*60)
    print("✅ M5 Sample Creation Complete")
    print(f"Final File: {OUTPUT_SALES_FILE}")
    print(f"Total Rows (Data Points): {total_output_rows}")
    print(f"Total Time: {end_time - start_time:.2f} seconds")
    print("Ready for PLX upload.")
    print("="*60)

if __name__ == '__main__':
    transform_m5_sales_sample()

In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
INPUT_FILE = 'exchange_rate.csv'
TRAIN_OUTPUT = 'Exchange_Rate_Train.csv'
TEST_OUTPUT = 'Exchange_Rate_Test.csv'

def process_exchange_rate_data():
    print("1. Loading Data...")
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found.")
        return

    # Load data
    # exchange_rate.csv often comes without headers (just 8 columns of rates) or with a date column.
    # We attempt to load it standardly first.
    try:
        df = pd.read_csv(INPUT_FILE)
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return

    # --- DATE HANDLING ---
    # Check if a date column exists. If not, we create a dummy time index.
    date_col = None
    for col in df.columns:
        if 'date' in col.lower() or 'timestamp' in col.lower():
            date_col = col
            break

    if date_col:
        print(f"   Detected date column: {date_col}")
        df['date'] = pd.to_datetime(df[date_col])
        if date_col != 'date':
            df.drop(columns=[date_col], inplace=True)
    else:
        print("   No date column found. Generating daily index starting 1990-01-01...")
        # Exchange Rate dataset (LSTNet version) is typically daily
        df['date'] = pd.date_range(start='1990-01-01', periods=len(df), freq='D')

    print(f"   Total Data Shape: {df.shape}")

    # --- STEP 2: SPLIT DATA (TIME BASED) ---
    print("2. Splitting Train/Test (Last 20% for Test)...")

    # Calculate split index based on rows (Time)
    total_rows = len(df)
    split_index = int(total_rows * 0.8) # 80% Train, 20% Test

    # Slice chronologically
    df_train_wide = df.iloc[:split_index].copy()
    df_test_wide = df.iloc[split_index:].copy()

    print(f"   Train Rows (Wide): {len(df_train_wide)}")
    print(f"   Test Rows (Wide):  {len(df_test_wide)}")

    # --- STEP 3: MELT AND PROCESS ---

    def process_split(df_wide, split_name):
        print(f"   Processing {split_name} split...")

        # Identify series columns (all columns except 'date')
        series_cols = [c for c in df_wide.columns if c != 'date']

        # Melt from Wide (Date, Rate1, Rate2...) to Long (Date, Series_ID, Value)
        df_long = df_wide.melt(
            id_vars=['date'],
            value_vars=series_cols,
            var_name='series_id',
            value_name='value'
        )

        # Sort for cleanliness
        df_long = df_long.sort_values(['series_id', 'date'])

        # Add metadata
        df_long['split_type'] = split_name

        return df_long

    # Process Train
    df_train_final = process_split(df_train_wide, 'TRAIN')

    # Process Test
    df_test_final = process_split(df_test_wide, 'TEST')

    # --- STEP 4: SAVING ---
    print(f"3. Saving Output Files...")

    print(f"   Saving {TRAIN_OUTPUT} ({len(df_train_final)} rows)...")
    df_train_final.to_csv(TRAIN_OUTPUT, index=False)

    print(f"   Saving {TEST_OUTPUT} ({len(df_test_final)} rows)...")
    df_test_final.to_csv(TEST_OUTPUT, index=False)

    print("\n✅ SUCCESS! Exchange Rate Data Reprocessed correctly.")

if __name__ == "__main__":
    process_exchange_rate_data()

In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
INPUT_FILE = 'ETTh1.csv'
TRAIN_OUTPUT = 'ETTh1_Train.csv'
TEST_OUTPUT = 'ETTh1_Test.csv'

def process_etth1_data():
    print("1. Loading Data...")
    if not os.path.exists(INPUT_FILE):
        print(f"Error: {INPUT_FILE} not found. Please upload ETTh1.csv")
        return

    # Load data
    df = pd.read_csv(INPUT_FILE)

    # --- DATE HANDLING ---
    # ETTh1 usually has a 'date' column.
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    else:
        print("   Warning: 'date' column not found. Looking for first column...")
        # Fallback: Assume first column is date
        date_col = df.columns[0]
        df.rename(columns={date_col: 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'])

    print(f"   Total Data Shape: {df.shape}")

    # --- STEP 2: SPLIT DATA (TIME BASED) ---
    print("2. Splitting Train/Test (Last 20% for Test)...")

    # Calculate split index based on rows (Time)
    total_rows = len(df)
    split_index = int(total_rows * 0.8) # 80% Train, 20% Test

    # Slice chronologically
    df_train_wide = df.iloc[:split_index].copy()
    df_test_wide = df.iloc[split_index:].copy()

    print(f"   Train Rows (Wide): {len(df_train_wide)}")
    print(f"   Test Rows (Wide):  {len(df_test_wide)}")

    # --- STEP 3: MELT AND PROCESS ---

    def process_split(df_wide, split_name):
        print(f"   Processing {split_name} split...")

        # Identify feature columns (all columns except 'date')
        # In ETTh1, these are typically: HUFL, HULL, MUFL, MULL, LUFL, LULL, OT
        feature_cols = [c for c in df_wide.columns if c != 'date']

        # Melt from Wide to Long
        df_long = df_wide.melt(
            id_vars=['date'],
            value_vars=feature_cols,
            var_name='series_id', # e.g., 'OT', 'HUFL'
            value_name='value'
        )

        # Sort by Series then Date
        df_long = df_long.sort_values(['series_id', 'date'])

        # Add metadata
        df_long['split_type'] = split_name

        return df_long

    # Process Train
    df_train_final = process_split(df_train_wide, 'TRAIN')

    # Process Test
    df_test_final = process_split(df_test_wide, 'TEST')

    # --- STEP 4: SAVING ---
    print(f"3. Saving Output Files...")

    print(f"   Saving {TRAIN_OUTPUT} ({len(df_train_final)} rows)...")
    df_train_final.to_csv(TRAIN_OUTPUT, index=False)

    print(f"   Saving {TEST_OUTPUT} ({len(df_test_final)} rows)...")
    df_test_final.to_csv(TEST_OUTPUT, index=False)

    print("\n✅ SUCCESS! ETTh1 Data Reprocessed correctly.")

if __name__ == "__main__":
    process_etth1_data()